In [ ]:
import os
import io
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload
from google.oauth2 import service_account
import json

class GoogleDriveUploader:
    def __init__(self, service_account_file):
        """
        구글 드라이브 업로더 초기화
        
        Args:
            service_account_file (str): 서비스 어카운트 JSON 파일 경로
        """
        self.service_account_file = service_account_file
        self.service = self._authenticate()
    
    def _authenticate(self):
        """서비스 어카운트로 구글 드라이브 API 인증"""
        try:
            # 서비스 어카운트 인증 정보 로드
            credentials = service_account.Credentials.from_service_account_file(
                self.service_account_file,
                scopes=['https://www.googleapis.com/auth/drive']
            )
            
            # 구글 드라이브 API 서비스 생성
            service = build('drive', 'v3', credentials=credentials)
            print("✅ 구글 드라이브 API 인증 성공")
            return service
            
        except Exception as e:
            print(f"❌ 인증 실패: {e}")
            return None
    
    def find_folder_id(self, folder_name, parent_folder_id='root'):
        """
        폴더명으로 폴더 ID 찾기
        
        Args:
            folder_name (str): 찾을 폴더명
            parent_folder_id (str): 부모 폴더 ID (기본값: 'root')
            
        Returns:
            str: 폴더 ID 또는 None
        """
        try:
            query = f"name='{folder_name}' and mimeType='application/vnd.google-apps.folder' and '{parent_folder_id}' in parents and trashed=false"
            
            results = self.service.files().list(
                q=query,
                fields="files(id, name)"
            ).execute()
            
            folders = results.get('files', [])
            
            if folders:
                folder_id = folders[0]['id']
                print(f"📁 폴더 '{folder_name}' 발견 (ID: {folder_id})")
                return folder_id
            else:
                print(f"📁 폴더 '{folder_name}'를 찾을 수 없습니다.")
                return None
                
        except Exception as e:
            print(f"❌ 폴더 검색 실패: {e}")
            return None
    
    def create_folder(self, folder_name, parent_folder_id='root'):
        """
        새 폴더 생성
        
        Args:
            folder_name (str): 생성할 폴더명
            parent_folder_id (str): 부모 폴더 ID (기본값: 'root')
            
        Returns:
            str: 생성된 폴더 ID 또는 None
        """
        try:
            folder_metadata = {
                'name': folder_name,
                'mimeType': 'application/vnd.google-apps.folder',
                'parents': [parent_folder_id]
            }
            
            folder = self.service.files().create(
                body=folder_metadata,
                fields='id'
            ).execute()
            
            folder_id = folder.get('id')
            print(f"📁 폴더 '{folder_name}' 생성 완료 (ID: {folder_id})")
            return folder_id
            
        except Exception as e:
            print(f"❌ 폴더 생성 실패: {e}")
            return None
    
    def upload_text_file(self, content, filename, folder_id=None):
        """
        텍스트 내용을 파일로 구글 드라이브에 업로드
        
        Args:
            content (str): 업로드할 텍스트 내용
            filename (str): 파일명
            folder_id (str): 업로드할 폴더 ID (기본값: None - 루트 폴더)
            
        Returns:
            str: 업로드된 파일 ID 또는 None
        """
        try:
            # 텍스트 내용을 BytesIO 객체로 변환
            file_content = io.BytesIO(content.encode('utf-8'))
            
            # 파일 메타데이터 설정
            file_metadata = {'name': filename}
            if folder_id:
                file_metadata['parents'] = [folder_id]
            
            # 미디어 업로드 객체 생성
            media = MediaIoBaseUpload(
                file_content, 
                mimetype='text/plain',
                resumable=True
            )
            
            # 파일 업로드 실행
            file = self.service.files().create(
                body=file_metadata,
                media_body=media,
                fields='id, name, webViewLink'
            ).execute()
            
            file_id = file.get('id')
            file_name = file.get('name')
            web_link = file.get('webViewLink')
            
            print(f"✅ 파일 업로드 성공!")
            print(f"   📄 파일명: {file_name}")
            print(f"   🆔 파일 ID: {file_id}")
            print(f"   🔗 링크: {web_link}")
            
            return file_id
            
        except Exception as e:
            print(f"❌ 파일 업로드 실패: {e}")
            return None
    
    def upload_local_file(self, file_path, folder_id=None, new_filename=None):
        """
        로컬 파일을 구글 드라이브에 업로드
        
        Args:
            file_path (str): 업로드할 로컬 파일 경로
            folder_id (str): 업로드할 폴더 ID (기본값: None - 루트 폴더)
            new_filename (str): 새 파일명 (기본값: None - 원본 파일명 사용)
            
        Returns:
            str: 업로드된 파일 ID 또는 None
        """
        try:
            if not os.path.exists(file_path):
                print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
                return None
            
            # 파일명 설정
            filename = new_filename if new_filename else os.path.basename(file_path)
            
            # 파일 메타데이터 설정
            file_metadata = {'name': filename}
            if folder_id:
                file_metadata['parents'] = [folder_id]
            
            # 파일 읽기 및 업로드
            with open(file_path, 'rb') as file_content:
                media = MediaIoBaseUpload(
                    io.BytesIO(file_content.read()),
                    mimetype='application/octet-stream',
                    resumable=True
                )
            
            # 파일 업로드 실행
            file = self.service.files().create(
                body=file_metadata,
                media_body=media,
                fields='id, name, webViewLink'
            ).execute()
            
            file_id = file.get('id')
            file_name = file.get('name')
            web_link = file.get('webViewLink')
            
            print(f"✅ 파일 업로드 성공!")
            print(f"   📄 파일명: {file_name}")
            print(f"   🆔 파일 ID: {file_id}")
            print(f"   🔗 링크: {web_link}")
            
            return file_id
            
        except Exception as e:
            print(f"❌ 파일 업로드 실패: {e}")
            return None

def main():
    """메인 함수 - 사용 예제"""
    
    # 서비스 어카운트 JSON 파일 경로
    SERVICE_ACCOUNT_FILE = '/Users/admin/DEVELOP/Y2025/GITHUB/my-talk/.key/service_account_key.json'
    
    # 업로드할 텍스트 내용
    text_content = """
안녕하세요! 이것은 구글 드라이브 업로드 테스트입니다.

이 파일은 파이썬 프로그램을 통해 자동으로 업로드되었습니다.
- 서비스 어카운트 인증 사용
- Google Drive API v3 활용
- 특정 폴더에 저장

작성 시간: 2025년 8월 23일
"""
    
    try:
        # 구글 드라이브 업로더 인스턴스 생성
        uploader = GoogleDriveUploader(SERVICE_ACCOUNT_FILE)
        
        if not uploader.service:
            print("❌ 업로더 초기화 실패")
            return
        
        # 1. 특정 폴더 찾기 (예: "업로드 테스트" 폴더)
        target_folder_name = "업로드 테스트"
        folder_id = uploader.find_folder_id(target_folder_name)
        
        # 2. 폴더가 없다면 생성
        if not folder_id:
            print(f"📁 '{target_folder_name}' 폴더가 없어서 생성합니다...")
            folder_id = uploader.create_folder(target_folder_name)
        
        if not folder_id:
            print("❌ 폴더 준비 실패")
            return
        
        # 3. 텍스트 내용을 파일로 업로드
        filename = f"테스트_문서_{int(time.time())}.txt"
        file_id = uploader.upload_text_file(
            content=text_content,
            filename=filename,
            folder_id=folder_id
        )
        
        # 4. 로컬 파일 업로드 예제 (선택사항)
        # local_file_path = "example.txt"
        # if os.path.exists(local_file_path):
        #     uploader.upload_local_file(local_file_path, folder_id)
        
        if file_id:
            print("\n🎉 모든 작업이 성공적으로 완료되었습니다!")
        else:
            print("\n❌ 일부 작업이 실패했습니다.")
            
    except Exception as e:
        print(f"❌ 프로그램 실행 중 오류 발생: {e}")

if __name__ == "__main__":
    import time
    main()